In [2]:
import os
import sys
sys.path.append(os.path.dirname(os.path.abspath('.')))

import pandas as pd
from utils import GCSHelper
import ast

# ITEM

## 0. Import Item Data

In [4]:
gcs_helper = GCSHelper('../keys/gcs_key.json')

item = gcs_helper.read_df_from_gcs('csv/item/maple_item_KMST_1149.csv')
item = item[item['overallCategory']=='Equip']
item = item[item['category']!='Mount'].reset_index(drop=True) # Mount는 펫이므로 삭제
item = item.drop(['overallCategory'], axis=1) # 모두 equip이므로 drop

## 1. Basic Instruction

In [174]:
item.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29740 entries, 0 to 29739
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   requiredJobs    29740 non-null  object
 1   requiredLevel   29740 non-null  object
 2   requiredGender  29740 non-null  object
 3   isCash          29740 non-null  bool  
 4   desc            29740 non-null  object
 5   id              29740 non-null  int64 
 6   name            29740 non-null  object
 7   category        29740 non-null  object
 8   subCategory     29740 non-null  object
 9   lowItemId       29740 non-null  int64 
 10  highItemId      29740 non-null  int64 
dtypes: bool(1), int64(3), object(7)
memory usage: 2.3+ MB


- NA가 NA로 들어가 있는 열은 없음
- int인 열은 있으나 Int로서의 의미는 없어보임
- columns
    - requiredJobs : 요구하는 직업
    - requiredLevel : 요구하는 레벨
    - requiredGender : 요구하는 성별
    - isCash : 캐시 아이템 여부 (bool)
    - desc : 설명
    - id : 아이템 아이디
    - name : 아이템 이름
    - OverallCategory
    - category
    - subCategory
    - lowItemId
    - highItemId

### 1-1. requiredJobs

In [175]:
print(item['requiredJobs'].nunique(), item['requiredJobs'].value_counts(), sep='\n')

12
['Beginner']                                            23483
['Warrior']                                              1892
['Thief']                                                1225
['Bowman']                                               1048
['Magician']                                             1035
['Pirate']                                                850
['Thief', 'Pirate']                                       117
None                                                       37
['Warrior', 'Thief']                                       27
['Warrior', 'Bowman', 'Thief']                             16
['Warrior', 'Magician']                                     7
['Warrior', 'Magician', 'Bowman', 'Thief', 'Pirate']        3
Name: requiredJobs, dtype: int64


- 이걸 feature로 쓰려면 각각을 다 column으로 빼줘서 boolean으로 넣어줘야 할 것 같음
- None이라는건 요구하는 직업이 없으니까 직업 상관 없이 다 입을 수 있다는 뜻??? -> 모르겠음
- 직업이 여기 나와 있는 Beginner, Warrior, Magician, Bowman, Thief, Pirate가 끝인가?
- Beginner가 직업인가??

In [176]:
# 리스트인 척 하는 string임 list로 변환해줘야 할 것 같으면 아래 코드 실행
# item['requiredJobs'] = item['requiredJobs'].apply(lambda x : ast.literal_eval(x))
item['requiredJobs'][0]

"['Beginner']"

### 1-2. requiredLevel

In [179]:
# requiredLevel이 None인걸 0과 동일하게 봐줘도 되나? 0이 따로 있는데
print(item['requiredLevel'].nunique(), item['requiredLevel'].value_counts(), sep='\n')

90
None    13979
0        7083
100       758
30        672
10        658
        ...  
83          1
73          1
52          1
92          1
14          1
Name: requiredLevel, Length: 90, dtype: int64


### 1-3. requiredGender

In [180]:
# 성별이 0부터 9까지 있다. 뭐라고 해석해야 할까? 도메인 널리지 불충분으로 해석 불가능
# 두개라면 그냥 T/F로 밀어버리려고 했는데 일단 후퇴한다.
print(item['requiredGender'].nunique(), item['requiredGender'].value_counts(), sep='\n')

9
2    15609
1     5656
0     5416
3     1802
5      604
4      534
6       94
9       16
8        9
Name: requiredGender, dtype: int64


### 1-4. isCash

In [181]:
# 깔끔하다 건들거 없다.
print(item['isCash'].nunique(), '\n', item['isCash'].value_counts())

2 
 True     20220
False     9520
Name: isCash, dtype: int64


### 1-5. desc -> 삭제

In [182]:
# 텍스트 쓰면 재밌을 것 같은데 29770개 중 desc 없는게 26000개 가량이므로 삭제해주자.
item['desc'].value_counts()['None']
item = item.drop(['desc'], axis=1)

### 1-6. id

In [183]:
# 다행인지 모든 Item id는 고유하다.
item['id'].nunique(), len(item['id'])

(29740, 29740)

### 1-7 name

In [184]:
# 불행히도 Item이름은 고유하지 않다.
item['name'].nunique(), len(item)

(21229, 29740)

In [185]:
# 동명이인인 아이템들은 왜 동명이인인지 살펴보자.
item['name'].value_counts().head(30)

분홍색 꽃무늬 튜브    35
은구슬 얼굴        32
차차 얼굴         32
랑드 얼굴         32
하늘 얼굴         32
총명한 얼굴        24
슈가 얼굴         24
도도한 얼굴        24
꽃분홍 튜브        21
혼돈의 포션        21
일리움 얼굴        17
뽀잉 얼굴         16
스윗하트 얼굴       16
알리샤 얼굴        16
숲의 요정 얼굴      16
사르르 얼굴        16
릴리 얼굴         16
랑랑 얼굴         16
폴암의 전사 얼굴     16
사연있는 얼굴       16
냐옹 얼굴         16
냉정한 얼굴        16
잊혀진 영웅 얼굴     16
드래곤 마스터 얼굴    16
빛과 어둠의 얼굴     16
괴도 팬텀 얼굴      16
빛나는 얼굴        16
산소같은 얼굴       16
인형 얼굴         16
불면증 얼굴        16
Name: name, dtype: int64

#### 동명이인 확인

In [186]:
# 이름이 같은 애들은 총 11564개
tmp = item['name'].value_counts()
tmp_df = item.copy()
tmp_df = tmp_df.merge(tmp.rename('count'), left_on='name', right_index=True)
tmp_df = tmp_df[tmp_df['count']>1]
print(len(tmp_df))
# 그 중 다른게 다 동일하고 아이템 Id만 다른 경우를 Drop하면 4675개가 된다
# 즉 아이템 아이디만 다른 경우가 6000개 가량 된다는 뜻으로 이 경우는 제거해주자
tmp_df = tmp_df.drop_duplicates(['requiredJobs', 'requiredLevel', 'requiredGender', 'isCash', 'name', 'category', 'subCategory', 'lowItemId', 'highItemId'])
print(len(tmp_df))

11564
4675


In [188]:
# set의 길이를 봄으로써 어디에서 갈리는지 확인해볼 것
tmp_set = tmp_df.groupby(['name']).agg(set).drop(['id'],axis=1)
# 이 중 이름이 None인 경우는 Drop
tmp_set = tmp_set.drop('None', axis=0)
for column in tmp_set.columns:
    tmp_set[f'{column}_len'] = tmp_set[column].apply(len).tolist()
    print(f"{column}_len---------", '\n', tmp_set[f'{column}_len'].value_counts(), sep='')
tmp_set = tmp_set.reset_index(drop=False)

requiredJobs_len---------
1    2985
2      37
5      25
6       2
3       2
4       1
Name: requiredJobs_len, dtype: int64
requiredLevel_len---------
1    2779
2     255
3      13
4       4
5       1
Name: requiredLevel_len, dtype: int64
requiredGender_len---------
1    2029
2     994
3      26
4       3
Name: requiredGender_len, dtype: int64
isCash_len---------
1    2992
2      60
Name: isCash_len, dtype: int64
category_len---------
1    3032
2      20
Name: category_len, dtype: int64
subCategory_len---------
1     3013
2       28
5        3
21       2
3        2
13       1
18       1
4        1
6        1
Name: subCategory_len, dtype: int64
lowItemId_len---------
1     2896
2      145
5        3
21       2
3        2
13       1
18       1
4        1
6        1
Name: lowItemId_len, dtype: int64
highItemId_len---------
1     2896
2      145
5        3
21       2
3        2
13       1
18       1
4        1
6        1
Name: highItemId_len, dtype: int64
count_len---------
1    3052
Name: 

In [189]:
# 일단 isCash가 True이면서 False인 애들을 살펴보자
tmp_set[tmp_set['isCash_len']==2]

,name,requiredJobs,requiredLevel,requiredGender,isCash,category,subCategory,lowItemId,highItemId,count,requiredJobs_len,requiredLevel_len,requiredGender_len,isCash_len,category_len,subCategory_len,lowItemId_len,highItemId_len,count_len
53,가면신사의 모자,{['Beginner']},"{10, 0, 13}","{3, 5, 2}","{False, True}",{Armor},{Hat},{1000000},{1010000},{3},1,3,3,2,1,1,1,1,1
69,갈색 단발 머리,{['Beginner']},{None},{2},"{False, True}",{Character},{Hair},{30000},{49999},{2},1,1,1,2,1,1,1,1,1
138,갈색 잊혀진 영웅 헤어,{['Beginner']},{None},{2},"{False, True}",{Character},{Hair},{30000},{49999},{2},1,1,1,2,1,1,1,1,1
173,거미줄 뾰족 모자,{['Beginner']},"{50, 0}","{3, 5}","{False, True}",{Armor},{Hat},{1000000},{1010000},{3},1,2,2,2,1,1,1,1,1
185,검은색 단발 머리,{['Beginner']},{None},{2},"{False, True}",{Character},{Hair},{30000},{49999},{2},1,1,1,2,1,1,1,1,1
412,꿀 송편 모자,{['Beginner']},"{33, 0}",{4},"{False, True}",{Armor},{Hat},{1000000},{1010000},{2},1,2,1,2,1,1,1,1,1
472,냉정한 얼굴,{['Beginner']},{None},"{1, 0}","{False, True}",{Character},{Face},{20000},{29999},{16},1,1,2,2,1,1,1,1,1
490,노란색 단발 머리,{['Beginner']},{None},{2},"{False, True}",{Character},{Hair},{30000},{49999},{2},1,1,1,2,1,1,1,1,1
558,노란색 잊혀진 영웅 헤어,{['Beginner']},{None},{2},"{False, True}",{Character},{Hair},{30000},{49999},{2},1,1,1,2,1,1,1,1,1
588,노블레스 화려한 로브,{['Beginner']},{0},{2},"{False, True}",{Armor},{Overall},{1050000},{1060000},{2},1,1,1,2,1,1,1,1,1
